<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [65]:
import qiskit
import numpy as np
from qiskit.aqua.operators import PauliExpectation, CircuitSampler, ExpectationFactory
from qiskit.aqua.operators import CircuitStateFn, StateFn, ListOp
from qiskit.aqua.operators import I, X, Y, Z

hamiltonian = (X^X^X^X^X^X^X) #+ (I^I^Z) + (I^Z^Z)
n_qubits = hamiltonian.to_circuit().num_qubits
print(n_qubits)

7


In [66]:
backend = qiskit.BasicAer.get_backend('statevector_simulator')
sampler = CircuitSampler(backend=backend)

[StateFn doc](https://qiskit.org/documentation/stubs/qiskit.aqua.operators.state_fns.StateFn.html) -> Seems to act like a statevector.

[ExpectationFactory doc](https://qiskit.org/documentation/stubs/qiskit.aqua.operators.expectations.ExpectationFactory.html)
 -> ExpectationFactory adds sampling noise.

In [67]:
# this leads to better results
# observable_meas = StateFn(hamiltonian, is_measurement=True)

# But why not use this like the qiskit's implementation of VQE
expectation = ExpectationFactory.build(operator=hamiltonian,backend=backend)
observable_meas = expectation.convert(StateFn(hamiltonian, is_measurement=True))

In [68]:
from qiskit.circuit.library import TwoLocal

def _get_ansatz(n_qubits):
    return TwoLocal(n_qubits, ['ry','rz'], 'cx', reps=1)

def get_num_parameters(n_qubits):
    return len(_get_ansatz(n_qubits).parameters)

def get_var_form(params, n_qubits=2):
    """Get an hardware-efficient ansatz for n_qubits
    given parameters.
    """
    
    # Define variational Form
    var_form = _get_ansatz(n_qubits)
    
    # Get Parameters from the variational form
    var_form_params = sorted(var_form.parameters, key=lambda p: p.name)
    
    # Check if the number of parameters is compatible
    assert len(var_form_params) == len(params), "The number of parameters don't match"
    
    # Create a dictionary with the parameters and values
    param_dict = dict(zip(var_form_params, params))
    
    # Assing those values for the ansatz
    wave_function = var_form.assign_parameters(param_dict)
    
    return wave_function

In [71]:
# n_qubits = 3
n_params = get_num_parameters(n_qubits)
params = [0]*n_params
wavefunction = get_var_form(params, n_qubits)

In [72]:
ansatz_circuit_op = CircuitStateFn(wavefunction)
expect_op = observable_meas.compose(ansatz_circuit_op).reduce()
sampled_expect_op = sampler.convert(expect_op)
means = np.real(sampled_expect_op.eval())

In [73]:
means

0.0